In [ ]:
# GET stock data

In [1]:
import json
import requests
import pandas as pd
import time
from fake_useragent import UserAgent
from datetime import datetime


ua = UserAgent()
headers = {"User-Agent":ua.random}

In [5]:
url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20200618&stockNo=2330&_=1592394271327"
res = requests.get(url,headers=headers)
json_stock = json.loads(res.content)

In [7]:
json_stock["data"]

[['109/06/01',
  '37,936,214',
  '11,205,227,094',
  '294.00',
  '296.50',
  '293.50',
  '295.50',
  '+3.50',
  '15,502'],
 ['109/06/02',
  '26,663,587',
  '7,908,476,089',
  '296.00',
  '297.50',
  '296.00',
  '296.50',
  '+1.00',
  '9,743'],
 ['109/06/03',
  '67,894,337',
  '20,354,971,677',
  '300.00',
  '301.00',
  '298.00',
  '301.00',
  '+4.50',
  '33,062'],
 ['109/06/04',
  '47,225,322',
  '14,414,983,532',
  '305.00',
  '306.00',
  '304.00',
  '306.00',
  '+5.00',
  '24,482'],
 ['109/06/05',
  '44,077,262',
  '13,655,618,646',
  '308.50',
  '312.00',
  '308.00',
  '311.50',
  '+5.50',
  '24,096'],
 ['109/06/08',
  '52,042,921',
  '16,500,836,851',
  '316.00',
  '319.00',
  '315.00',
  '318.00',
  '+6.50',
  '28,581'],
 ['109/06/09',
  '37,068,082',
  '11,746,874,522',
  '316.50',
  '319.00',
  '314.00',
  '319.00',
  '+1.00',
  '19,981'],
 ['109/06/10',
  '42,207,259',
  '13,566,733,306',
  '319.00',
  '324.00',
  '318.00',
  '322.50',
  '+3.50',
  '22,670'],
 ['109/06/11',
  '

In [ ]:
# for i in range(len(json_stock["data"])):
#     print(json_stock["data"][i][0])
#     print(json_stock["data"][i][1])
#     print("--------------")

In [4]:
for i in range(3600):
    res = requests.get(url,headers=headers)
    print("status: ",res.status_code," 次數 ：",i)
    time.sleep(3)
    

status:  200  次數 ： 0
status:  200  次數 ： 1
status:  200  次數 ： 2
status:  200  次數 ： 3
status:  200  次數 ： 4
status:  200  次數 ： 5
status:  200  次數 ： 6
status:  200  次數 ： 7
status:  200  次數 ： 8
status:  200  次數 ： 9
status:  200  次數 ： 10
status:  200  次數 ： 11
status:  200  次數 ： 12
status:  200  次數 ： 13
status:  200  次數 ： 14
status:  200  次數 ： 15
status:  200  次數 ： 16
status:  200  次數 ： 17
status:  200  次數 ： 18
status:  200  次數 ： 19
status:  200  次數 ： 20
status:  200  次數 ： 21
status:  200  次數 ： 22
status:  200  次數 ： 23
status:  200  次數 ： 24
status:  200  次數 ： 25
status:  200  次數 ： 26
status:  200  次數 ： 27
status:  200  次數 ： 28
status:  200  次數 ： 29
status:  200  次數 ： 30
status:  200  次數 ： 31
status:  200  次數 ： 32
status:  200  次數 ： 33
status:  200  次數 ： 34
status:  200  次數 ： 35
status:  200  次數 ： 36
status:  200  次數 ： 37
status:  200  次數 ： 38
status:  200  次數 ： 39
status:  200  次數 ： 40
status:  200  次數 ： 41
status:  200  次數 ： 42
status:  200  次數 ： 43
status:  200  次數 ： 44
status:  200  次數 ： 4

status:  200  次數 ： 361
status:  200  次數 ： 362
status:  200  次數 ： 363
status:  200  次數 ： 364
status:  200  次數 ： 365
status:  200  次數 ： 366
status:  200  次數 ： 367
status:  200  次數 ： 368
status:  200  次數 ： 369
status:  200  次數 ： 370
status:  200  次數 ： 371
status:  200  次數 ： 372
status:  200  次數 ： 373
status:  200  次數 ： 374
status:  200  次數 ： 375
status:  200  次數 ： 376
status:  200  次數 ： 377
status:  200  次數 ： 378
status:  200  次數 ： 379
status:  200  次數 ： 380
status:  200  次數 ： 381
status:  200  次數 ： 382
status:  200  次數 ： 383
status:  200  次數 ： 384
status:  200  次數 ： 385
status:  200  次數 ： 386
status:  200  次數 ： 387
status:  200  次數 ： 388
status:  200  次數 ： 389
status:  200  次數 ： 390
status:  200  次數 ： 391
status:  200  次數 ： 392
status:  200  次數 ： 393
status:  200  次數 ： 394
status:  200  次數 ： 395
status:  200  次數 ： 396
status:  200  次數 ： 397
status:  200  次數 ： 398
status:  200  次數 ： 399
status:  200  次數 ： 400
status:  200  次數 ： 401
status:  200  次數 ： 402
status:  200  次數 ： 403
status:  20

status:  200  次數 ： 718
status:  200  次數 ： 719
status:  200  次數 ： 720
status:  200  次數 ： 721
status:  200  次數 ： 722
status:  200  次數 ： 723
status:  200  次數 ： 724
status:  200  次數 ： 725
status:  200  次數 ： 726
status:  200  次數 ： 727
status:  200  次數 ： 728
status:  200  次數 ： 729
status:  200  次數 ： 730
status:  200  次數 ： 731
status:  200  次數 ： 732
status:  200  次數 ： 733
status:  200  次數 ： 734
status:  200  次數 ： 735
status:  200  次數 ： 736
status:  200  次數 ： 737
status:  200  次數 ： 738
status:  200  次數 ： 739
status:  200  次數 ： 740
status:  200  次數 ： 741
status:  200  次數 ： 742
status:  200  次數 ： 743
status:  200  次數 ： 744
status:  200  次數 ： 745
status:  200  次數 ： 746
status:  200  次數 ： 747
status:  200  次數 ： 748
status:  200  次數 ： 749
status:  200  次數 ： 750
status:  200  次數 ： 751
status:  200  次數 ： 752
status:  200  次數 ： 753
status:  200  次數 ： 754
status:  200  次數 ： 755
status:  200  次數 ： 756
status:  200  次數 ： 757
status:  200  次數 ： 758
status:  200  次數 ： 759
status:  200  次數 ： 760
status:  20

KeyboardInterrupt: 